<a href="https://colab.research.google.com/github/pmondal0402/FinancialEngineering/blob/master/Term_Structure_Lattice_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Week 5 
# Author : P. Mondal
# Date : Oct 20, 2020

In [5]:
import numpy as np

In [6]:
# Function to get short rate tree
def ShortRateTree(r00, qu, qd, n):
    shortRtree = np.zeros((n+1, n+1))
    
    # Setting r00 at i = 0 and j = 0
    shortRtree[0, 0] = r00
    for i in range(1, n+1):
      shortRtree[i, 0] = qd*shortRtree[i-1, 0] # ???? the code sol used qu 
      for j in range(1, n+1):
        shortRtree[i, j] = qu*shortRtree[i-1, j-1]
    return shortRtree



In [7]:
# Reproduced shortrate tree from class slides
test_shorttree = ShortRateTree(6, 1.25, 0.9, 5)
print(test_shorttree)

[[ 6.          0.          0.          0.          0.          0.        ]
 [ 5.4         7.5         0.          0.          0.          0.        ]
 [ 4.86        6.75        9.375       0.          0.          0.        ]
 [ 4.374       6.075       8.4375     11.71875     0.          0.        ]
 [ 3.9366      5.4675      7.59375    10.546875   14.6484375   0.        ]
 [ 3.54294     4.92075     6.834375    9.4921875  13.18359375 18.31054688]]


In [8]:
# Function to generate bond tree
def bondtree(bondMtry, r00, F, qu, qd, n):
  """Generates zero coupon bond tree"""
  q1 = q2 = 0.5 # Not sure 
  # Get shortrate tree corresponding to r00, qu, qd, n
  shortrT = ShortRateTree(r00, qu, qd, n)
  # print("shortT is\n", shortrT)

  bondT = np.zeros((bondMtry+1, bondMtry+1))
  # Generate bond tree : Going backward here
  for j in range(bondMtry+1):
    # At maturity all node states are set to facevalue
    bondT[bondMtry, j] = F
  # print(bondT)

  for i in range(bondMtry-1, -1, -1):
    for j in range(i+1):
      # Each i^th time has j = i + 1 nodes
      # print(i, j, bondT[i, j])
      bondT[i, j] = (1/(1+shortrT[i, j]))*(q1*bondT[i+1, j+1]
                                         + q2*bondT[i+1, j])
  return bondT


In [9]:
bondMtry = 4 # Bond maturity time
r00 = 6/100 # spot intereat rate at i = 0, j = 0
F = 100 # Facevalue 
qu = 1.25 # short interest rate up
qd = 0.9 # short interest rate down
n = 7 # number o short rate periods
bnd_tree = bondtree(bondMtry, r00, F, qu, qd, n)

In [10]:
print("Zero coupon price is ", bnd_tree[0, 0])

Zero coupon price is  77.217740328716


In [11]:
# --------
# Quiz 1 :
# --------
bondMtry = 10
r00 = 5/100
F = 100 
qu = 1.1
qd = 0.9
n = bondMtry + 2 # Getting shortrate tree for two more bond periods
bnd_tree_q1 = bondtree(bondMtry, r00, F, qu, qd, n)

In [12]:
print("Zero coupon price is ", bnd_tree_q1[0, 0])

Zero coupon price is  61.62195811754156


In [13]:
# ------
# Quiz 2
# ------
def bondForwrd(N, r00, qu, qd, F, bondMtry):
  """Looks like is the the ratio of price of considering bond maturity 
     and derivative maturity 
  """
  down = bondtree(N, r00, 1, qu, qd, n)
  up = bondtree(bondMtry, r00, F, qu, qd, n)
  return up[0, 0]/down[0, 0]

In [15]:
N = 4 # 
print("Price of Forward at t=0 :\n", bondForwrd(N, r00, qu, qd, F, bondMtry))

Price of Forward at t=0 :
 74.88484493844841
